# 05-Time Series Forecasting: Problem 2 

### - ARIMA 
### - Seasonal ARIMA (SARIMA)
### - Prophet

### 1. 모듈 불러오기

In [ ]:
!pip install pmdarima

import os
import datetime
import itertools

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt
import matplotlib
plt.style.use('seaborn-whitegrid')
import seaborn as sns

import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX 
from pmdarima.arima import auto_arima

!git clone https://github.com/hansam95/LG-Elec-Day3.git

### 2. 데이터 불러오기

### Wisconsin Employment Data
### 미국 위스콘신주의 취업건수(Employment) 예측을 위한 예측모델 학습


In [ ]:
data = pd.read_csv('/content/LG-Elec-Day3/data/Employment.csv')
data['month'] = pd.to_datetime(data['month'])
data = data.set_index('month')
data

### 3. Box-Jenkins ARIMA Procedure
-  3.1 Data Preprocessing
-  3.2 Identify Model to be Tentatively Entertainted
-  3.3 Estimate Parameters
-  3.4 Diagnosis Check
-  3.5 use Model to Forecast

### 3.1 Data Preprocessing

In [ ]:
data.plot(figsize=(12,4))

plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.legend('')

plt.title('Employment Data \n', fontsize=15)
plt.xlabel('\n Year', fontsize=13)
plt.ylabel('Employment \n', fontsize=13)
plt.tight_layout()
plt.show()

### Q: 시계열 분해 수행(Trend, Seasonal, Residual 그래프 확인하기)

In [ ]:
decomposition = sm.tsa.'''Answer'''(data['employment'],  model='additive')

fig = decomposition.plot()
fig.set_size_inches(10,10)
plt.show()

### 3.2 Identify Model to be Tentatively Entertainted

### Q : 학습 데이터와 테스트 데이터 분리 - train:test = 8:2 로 분리하기

In [ ]:
train, test = train_test_split(data, test_size='''Answer''', shuffle=False)

### Q : Autocorrelation , Partial Autocorrelation 확인하기

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,5))
fig.suptitle('Raw Data')
sm.graphics.tsa.'''Answer'''(train.values.squeeze(), lags=30, ax=ax[0])
sm.graphics.tsa.'''Answer'''(train.values.squeeze(), lags=30, ax=ax[1]);

### Q : 차분(first difference) 후 null 값 제거하기

In [ ]:
diff_train = train.copy()
diff_train = diff_train['employment'].'''Answer'''()
diff_train = diff_train.'''Answer'''()
print('####### Raw Data #######')
print(train)
print('### Differenced Data ###')
print(diff_train)

### Q : 그래프로 차분 전후 비교하기

In [ ]:
plt.figure(figsize=(12,8))

plt.subplot(211)
plt.plot('''Answer'''['employment'])
plt.legend(['Raw Data (Nonstationary)'])

plt.subplot(212)
plt.plot('''Answer''','orange')
plt.legend(['Differenced Data (Stationary)'])

plt.show()

###  차분 후 Autocorrelation , Partial Autocorrelation 확인하기

In [ ]:
fig, ax = plt.subplots(1,2,figsize=(10,5))
fig.suptitle('Differenced Data')
sm.graphics.tsa.plot_acf(diff_train.values.squeeze(), lags=30, ax=ax[0])
sm.graphics.tsa.plot_pacf(diff_train.values.squeeze(), lags=30, ax=ax[1]);

### 3.3 Estimate Parameters
### Q : 파라미터 조합 (1, 1, 0)으로 모델 학습 후 학습 결과 (Summary) 확인하기

In [ ]:
ARIMA_model = ARIMA(train.values, order=(1,1,0))
ARIMA_model_fit = ARIMA_model.'''Answer'''()
ARIMA_model_fit.'''Answer'''()

### 3.4.1 Diagnosis Check - ARIMA
### Q : ARIMA 모델 파라미터 탐색 범위 설정하기
 - p : 0 ~ 2
 - d : 1
 - q : 0 ~ 2

In [ ]:
print('Examples of parameter combinations for ARIMA...')
p = range('''Answer''')
d = range('''Answer''')
q = range('''Answer''')
pdq = list(itertools.product(p, d, q))
print(pdq)

### Q : 설정된  각 파라미터 조합으로 모델 학습하며 파라미터 조합마다 AIC 값 저장하기

In [ ]:
aic=[]
for i in '''Answer''':
    model = ARIMA(train.values, order=('''Answer'''))
    ARIMA_model_fit = model.fit()
    print(f'ARIMA: {i} >> AIC : {round(ARIMA_model_fit.aic,2)}')
    aic.append(round(ARIMA_model_fit.aic,2))

### Q : AIC 값이 가장 작은 최적의 파라미터 조합 찾기

In [ ]:
ARIMA_optimal = [(pdq[i], j) for i, j in enumerate(aic) if j == '''Answer'''(aic)]
ARIMA_optimal

### 가장 좋은 모델(최적 모델) 학습 결과 (Summary) 확인하기

In [ ]:
ARIMA_model_opt = ARIMA(train.values, order=ARIMA_optimal[0][0])
ARIMA_model_opt_fit = ARIMA_model_opt.fit()

ARIMA_model_opt_fit.summary()

### 3.5.1 Use Model to Forecast - ARIMA
### Q : 성능이 가장 좋은 모델로 미래 시점 예측하기

In [ ]:
ARIMA_prediction = ARIMA_model_opt_fit.forecast(len('''Answer''')).summary_frame()
ARIMA_test_pred = ARIMA_prediction['mean']
ARIMA_test_ub = ARIMA_prediction['mean_ci_upper']
ARIMA_test_lb = ARIMA_prediction['mean_ci_lower']
predict_index = list(test.index)

### Q : 모델 성능 확인하기

In [ ]:
print('ARIMA')
print('-'*30)
print(f'MSE: {np.round(mean_squared_error(test, ARIMA_test_pred), 2)}')
print(f'RMSE: {np.round(np.sqrt(mean_squared_error(test,ARIMA_test_pred)), 2)}')
print(f'MAE: {np.round(mean_absolute_error(test, ARIMA_test_pred), 2)}')
print(f'R2 score: {np.round(r2_score('''Answer''', '''Answer'''), 2)}')

### Q : 실제값과 예측값 그래프로 비교하기

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot('''Answer'''.index, '''Answer'''.values, label = 'Employment')
ax.plot(predict_index, '''Answer''', label = 'Prediction')
ax.vlines(datetime.datetime.strptime('1972-11-01','%Y-%m-%d'), 200, 450, linestyle='--', color='r', label='Start of Forecast')
ax.fill_between(predict_index, ARIMA_test_lb, ARIMA_test_ub, color = 'k', alpha = 0.1, label='0.95 Prediction Interval')
ax.legend(loc='upper left')
plt.suptitle(f'ARIMA {ARIMA_optimal[0][0]} Prediction Results (r2_score: {np.round(r2_score(test, ARIMA_test_pred), 2)}')
plt.show()

### 3.4.2 Diagnosis Check - SARIMA
### Q : SARIMA 모델 파라미터 탐색 범위 설정하기
 - p : 0 ~ 2
 - d : 1
 - q : 0 ~ 2
 - s : 12

In [ ]:
print('Examples of parameter combinations for Seasonal ARIMA...')
p = range('''Answer''')
d = range('''Answer''')
q = range('''Answer''')
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], '''Answer''') for x in list(itertools.product(p, d, q))]
print(seasonal_pdq)

### Q : 설정된  각 파라미터 조합으로 모델 학습하며 파라미터 조합마다 AIC 값 저장하기

In [ ]:
aic=[]
params=[]
for i in pdq:
    for j in '''Answer''':
        try:
            model = SARIMAX(train.values, order=('''Answer'''), seasonal_order = ('''Answer'''))
            model_fit = model.fit()
            print(f'SARIMA: {i}{j} >> AIC : {round(model_fit.aic,2)}')
            aic.append(round(model_fit.aic,2))
            params.append((i, j))  
        except:
            continue

### AIC 값이 가장 작은 최적의 파라미터 조합 찾기

In [ ]:
SARIMA_optimal = [(params[i], j) for i, j in enumerate(aic) if j == min(aic)]
SARIMA_optimal

### Q : 가장 좋은 모델(최적 모델) 학습 결과 (Summary) 확인하기

In [ ]:
SARIMA_model_opt = SARIMAX('''Answer'''.values, order=SARIMA_optimal[0][0][0], seasonal_order = SARIMA_optimal[0][0][1])
SARIMA_model_opt_fit = SARIMA_model_opt.fit()

SARIMA_model_opt_fit.summary()

### 3.5.2 Use Model to Forecast - SARIMA
### Q : 성능이 가장 좋은 모델로 미래 시점 예측하기

In [ ]:
SARIMA_prediction = SARIMA_model_opt_fit.get_forecast(len(test))
SARIMA_test_pred = '''Answer'''.predicted_mean
SARIMA_test_ub = '''Answer'''.conf_int()[:,0]
SARIMA_test_lb = '''Answer'''.conf_int()[:,1]
predict_index = list(test.index)

### Q : 모델 성능 확인하기

In [ ]:
print('SARIMA')
print('-'*30)
print(f'MSE: {np.round(mean_squared_error(test, SARIMA_test_pred), 2)}')
print(f'RMSE: {np.round(np.sqrt(mean_squared_error(test,SARIMA_test_pred)), 2)}')
print(f'MAE: {np.round(mean_absolute_error(test, SARIMA_test_pred), 2)}')
print(f'R2 score: {np.round('''Answer'''(test, SARIMA_test_pred), 2)}')

### 실제값과 예측값 그래프로 비교하기

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(data.index, data.values, label = 'Employment')
ax.plot(predict_index, SARIMA_test_pred, label = 'Prediction')
ax.vlines(datetime.datetime.strptime('1972-11-01','%Y-%m-%d'), 200, 450, linestyle='--', color='r', label='Start of Forecast')
ax.fill_between(predict_index, SARIMA_test_lb, SARIMA_test_ub, color = 'k', alpha = 0.1, label='0.95 Prediction Interval')
ax.legend(loc='upper left')
plt.suptitle(f'SARIMA {SARIMA_optimal[0][0][0]},{SARIMA_optimal[0][0][1]} Prediction Results (r2_score: {np.round(r2_score(test, SARIMA_test_pred), 2)})')
plt.show()

### 3.4.3 Diagnosis Check - auto_arima
### Q : auto_arima 모델 파라미터 탐색 범위 설정 후 모델 학습하기

In [ ]:
auto_arima_model = auto_arima('''Answer''', start_p=1, start_q=1,
                              max_p=3, max_q=3, m=12, seasonal=True,
                              d=1, D=1, 
                              max_P=3, max_Q=3,
                              trace=True,
                              error_action='ignore',  
                              suppress_warnings=True, 
                              stepwise=False)

### Q : 모델 학습 후 학습 결과 (Summary) 확인하기

In [ ]:
auto_arima_model.'''Answer'''()

### 3.5.3 Use Model to Forecast - auto_arima
### Q : auto_arima를 통해 선정된 성능이 가장 좋은 모델로 미래 시점 예측하기

In [ ]:
AUTO_ARIMA_prediction = auto_arima_model.predict(len('''Answer'''), return_conf_int=True)
AUTO_ARIMA_test_pred = AUTO_ARIMA_prediction[0]
AUTO_ARIMA_test_ub = AUTO_ARIMA_prediction[1][:,0]
AUTO_ARIMA_test_lb = AUTO_ARIMA_prediction[1][:,1]
predict_index = list(test.index)

### 모델 성능 확인하기

In [ ]:
print('AUTO ARIMA')
print('-'*30)
print(f'MSE: {np.round(mean_squared_error(test, AUTO_ARIMA_test_pred), 2)}')
print(f'RMSE: {np.round(np.sqrt(mean_squared_error(test,AUTO_ARIMA_test_pred)), 2)}')
print(f'MAE: {np.round(mean_absolute_error(test, AUTO_ARIMA_test_pred), 2)}')
print(f'R2 score: {np.round(r2_score(test, AUTO_ARIMA_test_pred), 2)}')

### Q : 실제값과 예측값 그래프로 비교하기

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
ax.plot(data.index, data.values, label = 'Employment')
ax.plot('''Answer''', AUTO_ARIMA_test_pred, label = 'Prediction')
ax.vlines(datetime.datetime.strptime('1972-11-01','%Y-%m-%d'), 200, 450, linestyle='--', color='r', label='Start of Forecast')
ax.fill_between(predict_index, '''Answer''', '''Answer''', color = 'k', alpha = 0.1, label='0.95 Prediction Interval')
ax.legend(loc='upper left')
plt.suptitle(f'SARIMA {auto_arima_model.order},{auto_arima_model.seasonal_order} Prediction Results (r2_score: {np.round(r2_score(test, AUTO_ARIMA_test_pred), 2)})')
plt.show()

# EOD